In [ ]:
import os

# Automatically set OMP_NUM_THREADS to all available cores
n_cores = os.cpu_count()
os.environ["OMP_NUM_THREADS"] = str(n_cores)
print(f"OMP_NUM_THREADS set to {n_cores}")

In [ ]:
from pysm3 import Sky
from pysm3 import units as u

import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

In [ ]:
import logging
log = logging.getLogger("pysm3")
log.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
log.addHandler(handler)

In [ ]:
def joint_limits(freq_maps, percentile=99.5):
    """Return symmetric color limits for a dictionary of Quantity maps."""
    stacked = np.concatenate([freq_maps[model][0].value.ravel() for model in freq_maps])
    vmax = np.percentile(np.abs(stacked), percentile)
    if vmax <= 0:
        vmax = np.max(np.abs(stacked))
    if vmax == 0:
        vmax = 1.0
    vmin = -vmax
    return vmin, vmax


def remove_monopole(component):
    """Subtract the mean signal and return the centered Quantity plus the monopole."""
    data = component.value
    mono = float(np.mean(data))
    centered = (data - mono) * component.unit
    return centered, mono

### Reference frequencies from the literature
- Birkinshaw (1999, *Physics Reports* 310, 97) derives the thermal SZ (tSZ) spectrum, showing the intensity dip near 128 GHz, a null at ≃217 GHz, and the positive maximum around 370 GHz, while the kinematic SZ (kSZ) contribution remains frequency-independent in thermodynamic CMB units.
- The review by Carlstrom, Holder & Reese (2002, *ARA&A* 40, 643) emphasises that multi-band SZ programs pair a low-frequency tSZ-dominated band (≲100 GHz) with bands on both sides of the 217 GHz null to isolate kSZ and foregrounds.
- Operational surveys such as the SPT-SZ camera explicitly observe at 95 GHz, 150 GHz, and 220 GHz to separate tSZ and kSZ signals (South Pole Telescope instrument description, retrieved Jan 2026).

Guided by those references, we probe 90, 150, 217, and 280 GHz below, bracketing the tSZ null and matching common survey bands.

## Thermal SZ Comparisons

Summary plots and statistics for the thermal component across WebSky and Agora realizations.

In [ ]:
models = ["tsz1", "tsz2", "tsz3", "tsz4"]
model_titles = {
    "tsz1": "WebSky tSZ",
    "tsz2": "Agora tSZ",
    "tsz3": "Agora tSZ (lensed)",
    "tsz4": "HalfDome tSZ",
}

skys = {model: Sky(nside=512, preset_strings=[model]) for model in models}

freqs_ghz = [90, 150, 217, 280]
maps = {
    freq: {model: skys[model].get_emission(freq * u.GHz) for model in models}
    for freq in freqs_ghz
}


In [ ]:
n_rows = len(freqs_ghz)
n_cols = len(models)
fig = plt.figure(figsize=(4.0 * n_cols, 4.2 * n_rows))
monopoles = {}
for row_idx, freq in enumerate(freqs_ghz, start=1):
    vmin, vmax = joint_limits(maps[freq], percentile=99.5)
    monopoles[freq] = {}
    for col_idx, model in enumerate(models, start=1):
        component = maps[freq][model][0]
        centered_map, mono = remove_monopole(component)
        monopoles[freq][model] = mono
        hp.mollview(
            centered_map,
            sub=(n_rows, n_cols, n_cols * (row_idx - 1) + col_idx),
            fig=fig,
            min=vmin,
            max=vmax,
            unit=str(component.unit),
            title="",
        )
        ax = plt.gca()
        ax.text(
            0.5,
            1.03,
            f"{model_titles[model]} @ {freq} GHz",
            transform=ax.transAxes,
            ha="center",
            va="bottom",
            fontsize=10,
            fontweight="bold",
        )

plt.subplots_adjust(hspace=0.45, top=0.88)

unit = str(next(iter(maps[freqs_ghz[0]].values()))[0].unit)
print("Monopoles removed (units in", unit, "):")
for freq in freqs_ghz:
    entries = [
        f"{model_titles[model]}: {monopoles[freq][model]:.3e}"
        for model in models
    ]
    print(f"  {freq} GHz -> " + ", ".join(entries))


In [ ]:
diff_stats = {}
fig = plt.figure(figsize=(6, 3.6 * len(freqs_ghz)))
for row_idx, freq in enumerate(freqs_ghz, start=1):
    diff = maps[freq]["tsz3"][0] - maps[freq]["tsz2"][0]
    diff_data = diff.value
    vmax = np.percentile(np.abs(diff_data), 99.5)
    if vmax <= 0:
        vmax = np.max(np.abs(diff_data))
    if vmax == 0:
        vmax = 1.0
    hp.mollview(
        diff_data,
        sub=(len(freqs_ghz), 1, row_idx),
        fig=fig,
        min=-vmax,
        max=vmax,
        unit=str(diff.unit),
        title=f"tSZ difference (tsz3 - tsz2) @ {freq} GHz",
    )
    diff_stats[freq] = {
        "mean": float(np.mean(diff_data)),
        "rms": float(np.sqrt(np.mean(diff_data**2))),
        "p99": float(np.percentile(np.abs(diff_data), 99.0)),
    }

plt.subplots_adjust(hspace=0.4)

print("Summary of tsz3 - tsz2 differences (units in", str(diff.unit), "):")
for freq in freqs_ghz:
    stats = diff_stats[freq]
    print(
        f"  {freq} GHz -> mean: {stats['mean']:.3e}, rms: {stats['rms']:.3e}, |diff|_99%: {stats['p99']:.3e}"
    )


In [ ]:
lmax = 3 * 512 - 1
ells = np.arange(lmax + 1)
d_ell_factor = ells * (ells + 1) / (2 * np.pi)

color_options = [
    "tab:red",
    "tab:purple",
    "tab:green",
    "tab:blue",
    "tab:orange",
]
model_colors = {
    model: color_options[idx % len(color_options)]
    for idx, model in enumerate(models)
}

fig, axes = plt.subplots(1, len(freqs_ghz), figsize=(4 * len(freqs_ghz), 4), sharey=True)
for ax, freq in zip(axes, freqs_ghz):
    for model in models:
        cls = hp.anafast(maps[freq][model][0].value, lmax=lmax)
        d_ell = d_ell_factor * cls
        ax.loglog(ells[1:], d_ell[1:], color=model_colors[model], label=model_titles[model])
    ax.set_title(f"{freq} GHz")
    ax.set_xlabel("$\\ell$")
    ax.grid(True, which="both", alpha=0.3)
axes[0].set_ylabel("$\\ell(\\ell+1)C_\\ell / 2\\pi$")
axes[-1].legend(loc="lower left", bbox_to_anchor=(1.02, 0.05))
fig.tight_layout()


In [ ]:
tsz_freq = 150
pair_labels = [
    ("tsz2", "tsz1"),
    ("tsz3", "tsz1"),
    ("tsz3", "tsz2"),
    ("tsz4", "tsz1"),
    ("tsz4", "tsz2"),
]
tsz_diff_stats = {}
fig = plt.figure(figsize=(6, 3.6 * len(pair_labels)))
for row_idx, (num, denom) in enumerate(pair_labels, start=1):
    diff = maps[tsz_freq][num][0] - maps[tsz_freq][denom][0]
    diff_data = diff.value
    vmax = np.percentile(np.abs(diff_data), 99.5)
    if vmax <= 0:
        vmax = np.max(np.abs(diff_data))
    if vmax == 0:
        vmax = 1.0
    hp.mollview(
        diff_data,
        sub=(len(pair_labels), 1, row_idx),
        fig=fig,
        min=-vmax,
        max=vmax,
        unit=str(diff.unit),
        title=f"tSZ difference ({model_titles[num]} - {model_titles[denom]}) @ {tsz_freq} GHz",
    )
    tsz_diff_stats[(num, denom)] = {
        "mean": float(np.mean(diff_data)),
        "rms": float(np.sqrt(np.mean(diff_data**2))),
        "p99": float(np.percentile(np.abs(diff_data), 99.0)),
    }

plt.subplots_adjust(hspace=0.4)

print("Summary of tSZ differences at", tsz_freq, "GHz (units in", str(diff.unit), "):")
for num, denom in pair_labels:
    stats = tsz_diff_stats[(num, denom)]
    print(
        f"  {model_titles[num]} - {model_titles[denom]} -> mean: {stats['mean']:.3e}, rms: {stats['rms']:.3e}, |diff|_99%: {stats['p99']:.3e}"
    )


## Kinematic SZ Comparisons

Parallel diagnostics for the kSZ templates so the two components stay visually separated.

In [ ]:
ksz_models = ["ksz1", "ksz2", "ksz3"]
ksz_titles = {
    "ksz1": "WebSky kSZ",
    "ksz2": "Agora kSZ",
    "ksz3": "Agora kSZ (lensed)",
}
ksz_freq = 150
ksz_skys = {model: Sky(nside=512, preset_strings=[model]) for model in ksz_models}
ksz_maps = {
    model: ksz_skys[model].get_emission(ksz_freq * u.GHz)
    for model in ksz_models
}
ksz_colors = {
    model: color_options[idx % len(color_options)]
    for idx, model in enumerate(ksz_models)
}
ksz_unit = str(next(iter(ksz_maps.values()))[0].unit)
print(
    f"Loaded kSZ templates at {ksz_freq} GHz (units: {ksz_unit}) -> "
    + ", ".join(ksz_titles[model] for model in ksz_models)
)

In [ ]:
fig = plt.figure(figsize=(4.0 * len(ksz_models), 4.0))
stacked = np.concatenate([ksz_maps[model][0].value.ravel() for model in ksz_models])
vmax = np.percentile(np.abs(stacked), 99.5)
if vmax <= 0:
    vmax = np.max(np.abs(stacked))
if vmax == 0:
    vmax = 1.0
vmin = -vmax
ksz_monopoles = {}
for col_idx, model in enumerate(ksz_models, start=1):
    component = ksz_maps[model][0]
    centered_map, mono = remove_monopole(component)
    ksz_monopoles[model] = mono
    hp.mollview(
        centered_map,
        sub=(1, len(ksz_models), col_idx),
        fig=fig,
        min=vmin,
        max=vmax,
        unit=str(component.unit),
        title="",
    )
    ax = plt.gca()
    ax.text(
        0.5,
        1.03,
        f"{ksz_titles[model]} @ {ksz_freq} GHz",
        transform=ax.transAxes,
        ha="center",
        va="bottom",
        fontsize=10,
        fontweight="bold",
    )

plt.subplots_adjust(top=0.85, wspace=0.25)

print("kSZ monopoles removed (units in", ksz_unit, "):")
for model in ksz_models:
    print(f"  {ksz_titles[model]}: {ksz_monopoles[model]:.3e}")

In [ ]:
ksz_diff = ksz_maps["ksz3"][0] - ksz_maps["ksz2"][0]
ksz_diff_data = ksz_diff.value
vmax = np.percentile(np.abs(ksz_diff_data), 99.5)
if vmax <= 0:
    vmax = np.max(np.abs(ksz_diff_data))
if vmax == 0:
    vmax = 1.0

fig = plt.figure(figsize=(6, 3.6))
hp.mollview(
    ksz_diff_data,
    sub=(1, 1, 1),
    fig=fig,
    min=-vmax,
    max=vmax,
    unit=str(ksz_diff.unit),
    title=f"kSZ difference ({ksz_titles['ksz3']} - {ksz_titles['ksz2']}) @ {ksz_freq} GHz",
)

ksz_23_stats = {
    "mean": float(np.mean(ksz_diff_data)),
    "rms": float(np.sqrt(np.mean(ksz_diff_data**2))),
    "p99": float(np.percentile(np.abs(ksz_diff_data), 99.0)),
}

print(
    "Summary of kSZ differences at",
    ksz_freq,
    "GHz (units in",
    str(ksz_diff.unit),
    "):",
)
print(
    f"  {ksz_titles['ksz3']} - {ksz_titles['ksz2']} -> mean: {ksz_23_stats['mean']:.3e}, rms: {ksz_23_stats['rms']:.3e}, |diff|_99%: {ksz_23_stats['p99']:.3e}",
)

In [ ]:
fig, ax = plt.subplots(figsize=(4.8, 4.0))
for model in ksz_models:
    cls = hp.anafast(ksz_maps[model][0].value, lmax=lmax)
    d_ell = d_ell_factor * cls
    ax.loglog(
        ells[1:],
        d_ell[1:],
        color=ksz_colors[model],
        label=ksz_titles[model],
    )

ax.set_title(f"{ksz_freq} GHz")
ax.set_xlabel("$\\ell$")
ax.set_ylabel("$\\ell(\\ell+1)C_\\ell / 2\\pi$")
ax.grid(True, which="both", alpha=0.3)
ax.legend(loc="lower left")
fig.tight_layout()